Собираем данные по автомоилям на авто.ру

In [1]:
pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13501 sha256=1c972644f70de16c9626cc49d2408b9b8a2c4fb4f987eaf0b5b8a2107b3172d0
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [2]:
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm


In [3]:
# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):
  response = requests.get(url) # , headers={'User-Agent': UserAgent().chrome}
  response.encoding ='utf8'   
    
  # Теперь создадим объект BeautifulSoup, указывая html парсер    
  page = BeautifulSoup(response.text, 'html.parser')
  data_dict = {}
  data_dict['car_url'] = url
  data_dict['parsing_unixtime'] = int(time.time())

  data_dict = {}
  for script in page.find_all("script"):
    # в a  ищем 'bodyType":' и берем нужное значение. Отсекаем начало, оставляя только содержимое словаря
    if 'bodyType":' in str(script): # ищем тип кузова 
      a = str(script)  # присваиваем a содержимое скрипта как строку    
      """Для среза используем шаблон: .* - сколько угодно индексов, ? до какого символа берем срез"""
      data_dict["bodyType"] = re.search(r'bodyType":".*?"', a)[0][11:].replace('"', '') # находим тэг, а рядом название (седан) 
      data_dict["brand"] = re.search(r'brand":".*?"', a)[0][8:].replace('"', '') #(bmw)
      data_dict["color"] = re.search(r'color":".*?"', a)[0][8:].replace('"', '') # цвет кузова
      data_dict["fuelType"] = re.search(r'fuelType":".*?"', a)[0][11:].replace('"', '') # тип топлива
      data_dict["modelDate"] = re.search(r'modelDate":.*?"', a)[0][11:].replace(',"', '') # модельный год
      data_dict["productionDate"] = re.search(r'productionDate":.*?"', a)[0][16:].replace(',"', '') # год продажи
      data_dict["numberOfDoors"] = re.search(r'numberOfDoors":.*?"', a)[0][15:].replace(',"', '') # количестов дверей
      data_dict["vehicleConfiguration"] = re.search(r'vehicleConfiguration":".*?"', a)[0][23:].replace('"', '') #конфиг кузова
      data_dict["vehicleTransmission"] = re.search(r'vehicleTransmission":".*?"', a)[0][22:].replace('"', '') # type gearbox
      data_dict["enginePower"] = re.search(r'enginePower":".*?"', a)[0][14:].replace(' N12"', '') #  Power horse
      data_dict["engineDisplacement"] = re.search(r'engineDisplacement":".*?"', a)[0][21:] # Объем двс
      try:
        data_dict["price"] = re.search(r'price":.*?"', a)[0][8:].replace(',"', '') #    
      except:
        data_dict["price"] = None 
      data_dict["description"] = re.search(r'description":".*?"', a)[0][14:].replace('"', '')
    
    if '{"mileage":' in str(script): # ищем пробег
      a = str(script) 
      data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

    if 'complectation":{"id"' in str(script): # ищем комплектацию
      a = str(script)
      try:
        data_dict['complectation_dict'] = re.search(r'equipment":{".*?}', a)[0][13:].replace('":true', '').replace('}', '')
      except:
        data_dict['complectation_dict'] = None
    if 'equipment":{' in str(script):
      a = str(script) 
      data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:].replace('":true', '').replace('}', '')
    if '"model_info":' in str(script): # Название модели
      a = str(script)  # присваиваем a содержимое скрипта как строку

      data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')
    if 'super_gen":{' in str(script):
        a = str(script)  # присваиваем a содержимое скрипта как строку
        data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

    if 'vendor":"' in str(script): # Страна производитель 
        a = str(script)  # присваиваем a содержимое скрипта как строку
        data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')

  span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})

  for i,tag in enumerate (span_CardInfoRow__cell):
    if tag.text == "Владельцы":
      data_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0владелец', u'') # в  конце заменяем юникодовский пробел

    if tag.text == "Владение":
      data_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

    if tag.text == "ПТС":
      data_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

    if tag.text == "Привод":
      data_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

    if tag.text == "Руль":
      data_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

    if tag.text == "Состояние":
      data_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

    if tag.text == "Таможня":
      data_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
  return data_dict
      
# Функция по созданию списка ссылок links_list  на объявления о продаже автомобилей  
def extraction_links(url):
    links_list =[] 
    response = requests.get(url)  
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list    
    
df = pd.DataFrame() # инициализируем итоговый датафрейм 

url_link_list = [] # список страниц по годам и номерам от 1 до 99

ranges = [range(2014, 2019), range(1, 100)] # указываем года и диапазон страниц, которые будем парсить 

# index[0] - year, index[1] - page 
for index in itertools.product(*ranges):
    # формируем ссылки страницы со списками объявлений
    url_links  = (f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    url_link_list.append(url_links) # заносим их в список +
    

In [4]:
len(url_link_list)

1089

In [ ]:
links_list = [] # список списков ссылок на объявления c одной страницы таблицы объявлений
count_false = 0 # количество неудач
for url_links in tqdm(url_link_list):
# извлекаем в links_list список ссылок на объявления
  try:
      links_list += extraction_links(url_links)    
  except:
    count_false +=1
print(f'Количество неудачных страниц {count_false} из {len(links_list)} страниц')



In [ ]:
len(links_list)

In [3]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  2


In [ ]:
ads_dict_list = [] # список словарей содержимого объявлений
df = pd.DataFrame() # инициализируем итоговый датафрейм 
count_false = 0
try:
  for links in tqdm(links_list):
  получаем данные ввиде словаря 
    ads_dict_list = parsing_page_one_ad(links)
    df = df.append(ads_dict_list, ignore_index=True)
except:
  count_false +=1
df = df.append(ads_dict_list, ignore_index=True)
print(f'Количество неудачных страниц {count_false} из {len(links_list)} страниц')

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv('_auto_ru_2008_2018.csv', encoding = 'utf-8', index=False) # записываем содержимое датафрейма в файл

In [ ]:
# Список взят из Kaggle test
column = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_name',
        'numberOfDoors',
       'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня']

In [ ]:
list(set(column) - set(data_dict.keys())) # смотрим какие еще колонки не догрузили 
#=> удалим из списка некоторые колонки model_info, 'parsing_unixtime',  'super_gen', 'priceCurrency'
# model_info, name, sell_id

['image', 'car_url']

In [ ]:
df

,bodyType,brand,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,model_name,numberOfDoors,price,productionDate,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня
0,внедорожник 5 дв.,LEXUS,чёрный,"cruise-control,""roller-blind-for-rear-window,""...",Автомобиль в отличном состоянии! 1 Собственник...,"4.5 LTR""",272,"{""cruise-control,""roller-blind-for-rear-window...",дизель,80000,2015,LX,5,449000,2018,"{""id"":""20673952"",""name"":""III Рестайлинг 2"",""ru...",ALLROAD_5_DOORS AUTOMATIC 4.5,автоматическая,JAPANESE,1,Оригинал,полный,Левый,Не требует ремонта,Растаможен
1,седан,TOYOTA,белый,"cruise-control,""asr,""start-stop-function,""airb...",24 июня НОЧЬ ПРОДАЖ автомобилей с пробегом в Р...,"2.5 LTR""",181,"{""cruise-control,""asr,""start-stop-function,""ai...",бензин,54000,2017,CAMRY,4,950000,2018,"{""id"":""21110739"",""name"":""VIII (XV70)"",""ru_name...",SEDAN AUTOMATIC 2.5,автоматическая,JAPANESE,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,хэтчбек 5 дв.,KIA,белый,"asr,""esp,""airbag-driver,""isofix,""usb,""airbag-p...",ЛОТ: 01260582\nFAVORIT MOTORS KIA Север\nДанны...,"1.3 LTR""",84,"{""asr,""esp,""airbag-driver,""isofix,""usb,""airbag...",бензин,61473,2017,PICANTO,5,89000,2018,"{""id"":""20933523"",""name"":""III"",""ru_name"":""3"",""y...",HATCHBACK_5_DOORS AUTOMATIC 1.3,автоматическая,KOREAN,1,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,седан,FORD,серебристый,"multi-wheel,""rear-camera","Продаю Ford Fusion, только из США. Снега и ре...","1.5 LTR""",184,"{""multi-wheel,""rear-camera",бензин,30000,2016,FUSION_NA,4,400000,2018,"{""id"":""20910388"",""name"":""II Рестайлинг"",""ru_na...",SEDAN AUTOMATIC 1.5,автоматическая,AMERICAN,1,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,седан,MERCEDES,чёрный,"asr,""front-seats-heat-vent,""start-stop-functio...",Не соглашайтесь на меньшее. Июнь — время выгод...,"2.9 LTR""",249,"{""asr,""front-seats-heat-vent,""start-stop-funct...",дизель,71798,2017,CLS_KLASSE,4,674000,2018,"{""id"":""21190300"",""name"":""III (C257)"",""ru_name""...",SEDAN AUTOMATIC 2.9,автоматическая,EUROPEAN,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен
